# Daten untersuchen mit Pandas und Python 

## Vorbereitungen

### auf binder
Alle notwendigen Pakete werden von binder durch die Liste in `./environment.yml` automatisch geladen. Nichts weiter zu tun!


### für lokale Installationen
Mit pip package manager:
`$ python3 -m pip install requests pandas matplotlib`

Mit conda package manager:
`$ conda install requests pandas matplotlib`

## Hallo Pandas!

![Pandas](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Grosser_Panda.JPG/320px-Grosser_Panda.JPG)

### Daten holen
#### a) Daten holen bei binder-Ausführung
In der Datei `./postBuild` wird automatisch von binder mit dem Tool wget die Datei aus dem GitHub Repository in die binder Ausführungsumgebung geholt. Alles erledigt.

Inhalt von `./postBuild`:
```console 
wget -q -O nba_all_elo.csv "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv"
```

#### b) Daten holen bei lokaler Ausführung
Mit dem Requests-Paket können wir mit Webservern über HTTP kommunizieren.
Lassen Sie uns die Datei herunterladen und in das aktuelle Arbeitsverzeichnis speichern.

In [ ]:
import requests

download_url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv"
target_csv_path = "nba_all_elo.csv"

response = requests.get(download_url)
response.raise_for_status()
with open(target_csv_path, "wb") as f:
    f.write(response.content)
print("Download ready.")

### Daten aus der CSV-Datei lesen

Nun benutzen wir pandas um die [CSV-Datei](https://de.wikipedia.org/wiki/CSV_(Dateiformat)) einzulesen. Damit das funktioniert muss vorher noch das pandas-Paket mit dem `import`-Befehl unter dem Namen `pd` verfügbar gemacht werden.

Dabei wird ein sogenannter [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) mit dem Namen `nba` erzeugt. Vereinfacht kann man sich darunter eine Tabelle mit Spalten und Zeilen vorstellen. DataFrames sind **die** zentralen Datentypen in pandas.   

Mit dem Befehl `type()` kann der Datentyp eines jeden Python-Objekts ausgegeben werden. 

In [ ]:
import pandas as pd
nba = pd.read_csv("nba_all_elo.csv")
type(nba)
# Expected:
# <class 'pandas.core.frame.DataFrame'>

### DataFrame untersuchen

Die Funktion `len()` aus dem Python Standard gibt bei pandas DataFrames wie erwartet die Anzahl der Zeilen aus.

In [ ]:
len(nba)
# Expected:
# 126314

Die (pandas-)Funktion* (`shape()`) gibt die "Form" das Frames aus, also die Zeilen- und Spaltenanzahl. 

*: Es ist eigentlich eine Funktion des des numpy-Pakets. Da pandas aber zu weiten Teilen auf numpy basiert, gibt es diese Unschärfe an vielen Stellen. Wenn Funktionen explizit in pandas verfügbar sind, ist hier auf die pandas API verwiesen ([vgl. pandas Dokumentation zu shape()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html?highlight=shape#pandas.DataFrame.shape)).

In [ ]:
nba.shape
# Expected:
# (126314, 23)

Einen schnellen Blick in den DataFrame erhält man mit `head()`. Wird kein Parameter übergeben, so werden fünf Zeilen (und die Spaltenüberschriften) ausgegeben.

In [ ]:
nba.head()

Bei sehr breiten Tabellen werden je nach Breite des Bildschirms automatisch Spalten bei der Anzeige ausgeblendet. Dies ist oben durch "..." in der Ausgabe gekennzeichnet.

Soll explizit die Anzahl der anzuzeigenden Spalten begrenzt oder wie im Beispiel unten aufgehoben werden (hier `None`), so kann die über setzen der Optionen des DataFrames mir `set_option()` geschehen (vgl. [Liste aller möglichen Optionen](https://pandas.pydata.org/docs/reference/api/pandas.set_option.html?highlight=set_option#pandas.set_option)).

Nun wird die gesamte Breite des DataFrames ausgegeben.

In [ ]:
pd.set_option("display.max.columns", None)
nba.head()

Die Option `display_precision`hilft die ANzeige von Gleitpunktzahlen lesbarer zu gestalten.

Analog zu `head()` liefert `tail()` das Ende, also die letzten Teilen eines Frames.

In [ ]:
pd.set_option("display.precision", 2)
nba.tail()

### Aufgabe

Geben Sie die letzten drei Zeilen des DataFrames aus.

## Den Inhalt des DataFrames kennenlernen

### Datentypen mit `.info()` anzeigen

Die `info()`-Funktion zeigt für alle Spalten des DataFrame die verwendeten Datentypen an. Hier hat pandas beim Import der CSV-Datei zu Beginn alle Spalten analysiert un einen mehr oder weniger geeigneten Datentyp ausgewählt. 
* `int64` ist ein ganzzahliger numerischer 64-bit Datentyp 
* `float64`ist eine 64-bit Gleitpunktzahl
* `object` ist der allen Datentypen zugrundeliegende Basisdatentyp in Pythons Datentypenhierarchie. 

Man erkennt dass für alle numerischen Typen abhängig vom Vorhandenseins eines Kommas (oder Punkts) dezidiert entweder `int64` oder `float64` festgelegt, für alle anderen Spalten jedoch der sehr allgemeine Datentyp `object`. Folgen und Maßnahmen dazu lernen Sie im weiteren Verlauf kennen.

Interssant ist auch die Spalte `Non-Null-Count` in der Ausgabe. Alle Spalten haben für jede Zeile Werte, bis auf die Spalte `notes`, die nur in 5424 der 126314 Zeilen gefüllt ist.

In [ ]:
nba.info()

### (Mini-)Statistik mit  `.describe()`

Die Funktion `describe()` generiert eine zusammenfassende Statistik aller *numerischen* Spaten des DataFrames. Fehlende Werte (NaNs) in den Spalten werden dabei ignoriert.  

* count: Anzahl der Zeilen, die gefüllt sind (also nicht NaN)
* mean: Arithmetisches Mittel
* std: Standardabweichung, ugs. "Durchschnittliche Entfernung der Werte vom Mittel"
* min/max: Minimum und Maximum
* 25%/50%/75% Perzentile: Quantile (Bsp für 25% Perzentil: 25% der Werte sind kleiner als X)

In [ ]:
nba.describe()

Eine Minimal-Statistik für nicht-numerische Typen erhält man, wenn man describe den parameter `include=object` übergibt.

In [ ]:
nba.describe(include=object)

### Explorative Datenanalyse mit Pandas (ein Anfang...)

Mit dem Index-Operator `[]` greifen wir auf die Spalte mit dem Label team_id zu. Die Funktion `value_counts()` berechnet die Anzahl der der Sätze mit gleichem Inhalt. In unserem Beispiel wird dadurch die Anzahl der Heimspiele für jede Mannschaft ermittelt.

In [ ]:
nba["team_id"].value_counts()
# Expected:
# BOS    5997
# NYK    5769
# LAL    5078

# SDS      11

Benutzen wir die Spalte `fran_id` so wird das Gleiche für den Namen des Teams ermittelt. Beachten Sie dass die Boston Celtics (BOS) der Wert übereinstimmt, die Lakers (LAL) aber abweichende Werte haben. Wie kann das sein?  

In [ ]:
nba["fran_id"].value_counts()
# Expected:
# Lakers          6024
# Celtics         5997
# Knicks          5769

# Falcons           60
# Name: fran_id, dtype: int64

An  die Funktion `loc[]` kann als erstes Argument eine boolsche Bedingung (= Filter) und als zweites Argument eine Liste an auszugebenden Spalten übergeben werden.
Im Beispiel wird dadurch die Ermittlung der Spielanzahl nur für Zeilen mit dem Teamnamen `Lakers` durchgeführt. 

Es gibt wohl noch ein zweites Team mit dem selben Namen.

In [ ]:
nba.loc[nba["fran_id"] == "Lakers", ("team_id", "fran_id")].value_counts()
# Expected:
# LAL    5078
# MNL     946
# Name: team_id, dtype: int64

Lassen Sie uns die Spiele der Lakers untersuchen. Dazu wandeln wir zuerst die Spalte mit dem Spieldatum in den richtigen Datentyp um. Bisher war die Spalte vom allgemeinen Typ object und somit nicht für Datumsoperationen nutzbar.

In [ ]:
nba["date_played"] = pd.to_datetime(nba["date_game"])

Das allererste Spiel des Teams MNL:

In [ ]:
nba.loc[nba["team_id"] == "MNL", "date_played"].min()
# Expected:
# Timestamp('1948-11-04 00:00:00')

Das letzte Spiel des Teams MNL:

In [ ]:
nba.loc[nba["team_id"] == "MNL", "date_played"].max()
# Expected:
# Timestamp('1960-03-26 00:00:00')

Beide Aggregatsfunktionen `min` und `max` können wir auch in einem Durchlauf mit der funktion `agg()` ausführen.

In [ ]:
nba.loc[nba["team_id"] == "MNL", "date_played"].agg(("min", "max"))
# Expected:
# min   1948-11-04
# max   1960-03-26
# Name: date_played, dtype: datetime64[ns]

Die MNL Lakers (Minneapolis Lakers) waren also nur bis in die 60er Jahre aktiv. Danach ist das Team (bzw. das Franchise) von Minnesota nach Los Angeles umgezogen. [https://de.wikipedia.org/wiki/Los_Angeles_Lakers](https://de.wikipedia.org/wiki/Los_Angeles_Lakers)

#### Aufgabe

Wie viele Punkte haben die Boston Celtics in allen Spielen dieses data sets erzielt?

## Pandas Datenstrukturen kennenlernen

### `Series`: Die "Basis"-Datenstruktur in Pandas

Man erzeugt einen neue Series mit dem `Series()`-Befehl. Hier können z.B. Python Listen als Argument übergeben werden. Im Beispiel unten wird eine Reihe von Umsätzen erzeugt.

In [ ]:
revenues = pd.Series([5555, 7000, 1980])
revenues
# Expected:
# 0    5555
# 1    7000
# 2    1980
# dtype: int64

Im Ergebnis sind zwei Spalten zu erkennen: die eigentlichen Umsätze und eine Nummerierung für jede Zeile. Zugriff auf die Werte dieser series erhält man mit `revenue.values`. Das Ergebnis ist ein Python Array.

In [ ]:
revenues.values
# Expected:
# array([5555, 7000, 1980])

In [ ]:
type(revenues.values)
# Expected:
# <class 'numpy.ndarray'>

Die Nummerierung der Sätze ist der Index über den auf die Werte zugegriffen werden kann. [Pandas RangeIndex](https://pandas.pydata.org/docs/reference/api/pandas.RangeIndex.html)

In [ ]:
revenues.index
# Expected:
# RangeIndex(start=0, stop=3, step=1)

Indizes zu Series können auch explizit vergeben werden. Im Beispiel unten wird der Series ein alphanumerischer Index zugewiesen der die Namen der Städte als "Labels" für die Umsätze festlegt.

In [ ]:
city_revenues = pd.Series(
    [4200, 8000, 6500],
    index=["Amsterdam", "Toronto", "Tokyo"]
)
city_revenues
# Expected:
# Amsterdam    4200
# Toronto      8000
# Tokyo        6500
# dtype: int64

In [ ]:
city_revenues.index

Wird ein Python dictionary (andernorts auch Hash genannt, also eine Menge an Schlüssel+Wert Paaren) als Datenelement bei der Initialisierung der Series verwendet, so wird automatisch ein Index in der Reihenfolge der Schlüsselwerte erzeugt.

In [ ]:
city_employee_count = pd.Series({"Amsterdam": 5, "Tokyo": 8})
city_employee_count
# Expected:
# Amsterdam    5
# Tokyo        8
# dtype: int64

Die dictionary Methode `city_employee_count.keys()` führt bei series zum gleichen Ergebnis wie `city_employee_count.index`.

In [ ]:
city_employee_count.keys()
# Expected:
# Index(['Amsterdam', 'Tokyo'], dtype='object')

Series können wie Listen und andere Sequence-artige Typen auf Vorhandensein von Elementen überprüft werden.

In [ ]:
"Tokyo" in city_employee_count
# Expected:
# True

In [ ]:
"New York" in city_employee_count
# Expected:
# False

### `DataFrame`: Pandas' beliebteste Datenstruktur

DataFrames sind mehrdimensionale Datenstrukturen die aus mehreren Datenreihen (-->Series) mit entsprechenden Labels (also Spaltenüberschriften) und einem Index bestehen. In unserem Beispiel verbinden wir die beiden obigen Series Objekte zu einem gemeinsamen DataFrame. Der Inder wird über das verschmelzen der beiden Indizes der ursprünglichen Series gebildet. Liegt zu einem der Indizes nicht in jeder Series ein Wert vor, so wird mit `NaN` aufgefüllt.

In [ ]:
city_data = pd.DataFrame({
    "revenue": city_revenues,
    "employee_count": city_employee_count
})
city_data
# Expected:
#               revenue         employee_count
# Amsterdam     4200            5.0
# Tokyo         6500            8.0
# Toronto       8000            NaN

In [ ]:
city_data.index
# Expected:
# Index(['Amsterdam', 'Tokyo', 'Toronto'], dtype='object')

Die Werte sind nun keine eindimensionalen Arrays mehr sondern, wie erwartet, zweidimensional.

In [ ]:
city_data.values
# Expected:
# array([[4.2e+03, 5.0e+00],
#        [6.5e+03, 8.0e+00],
#        [8.0e+03,     nan]])

Lassen wir uns die "Achsen" dieser zweidimensionalen Datenstruktur mit `city_data.axes` ausgeben, so sieht man, dass auch für die Spaltenüberschriften, also für unser zwei Variablen (Umsatz und # Mitarbeiter), ein Index gebildet wurde.
Diese beiden Indizes sind ihrerseits in einem Array verpackt (man bemerke die eckigen Klammern ganz außen) und so kann wie gewohnt mit `[0]` und `[1]` über die Position im Array auf die einzelnen Objekte zugegriffen werden.

In [ ]:
city_data.axes
# Expected:
# [Index(['Amsterdam', 'Tokyo', 'Toronto'], dtype='object'),
# Index(['revenue', 'employee_count'], dtype='object')]

In [ ]:
city_data.axes[0]
# Expected:
# Index(['Amsterdam', 'Tokyo', 'Toronto'], dtype='object')

In [ ]:
city_data.axes[1]
# Expected:
# Index(['revenue', 'employee_count'], dtype='object')

`Achtung`

Es gibt auch Unterschiede wie diese beiden Indizes auf DataFrame ebene angesprochen werden. 

Die Funktion `keys()` liefert den "Spaltenindex", das Attribut `.index` liefert hingegen den "Zeilenindex".
Wird mit dem `in` Operator direkt der DataFrame durchsucht, so wird nur in den Spaltenüberschriften gesucht.

In [ ]:
city_data.keys()
# Expected:
# Index(['revenue', 'employee_count'], dtype='object')

In [ ]:
city_data.index

In [ ]:
"Amsterdam" in city_data.index

In [ ]:
"Amsterdam" in city_data
# Expected:
# False

In [ ]:
"revenue" in city_data
# Expected:
# True

#### Aufgabe

* Zeigen Sie die Achsen und den index des `nba` datasets an.
* Überprüfen Sie, ob die Spalte "points" existiert. Oder hieß die vielleicht "pts"?

## Zugriff auf Elemente für Python `list` und Pandas `Series` Objekte: Gemeinsamkeiten und Unterschiede

### Der Python Index Operator `[]`

In [ ]:
city_revenues
# Expected:
# Amsterdam    4200
# Toronto      8000
# Tokyo        6500
# dtype: int64

Zugriff über Index-Wert:

In [ ]:
city_revenues["Toronto"]
# Expected:
# 8000

Zugriff über Index-Position:

In [ ]:
city_revenues[1]
# Expected:
# 8000

Die Position `[-1]` liefert das letzte Element.

In [ ]:
city_revenues[-1]
# Expected:
# 6500

Bereiche können einfach über`[n:m]` selektiert werden. n ist dabei immer eingeschlossen und m ausgeschlossen. Auch einseitig begrenzte Intervalle sind durch Weglassen einer Grenze möglich.

In [ ]:
city_revenues[1:]
# Expected:
# Toronto    8000
# Tokyo      6500
# dtype: int64

In [ ]:
city_revenues["Toronto":]
# Expected:
# Toronto    8000
# Tokyo      6500
# dtype: int64

### Zugriff auf `Series`' Elemente via `.loc` und `.iloc`

In [ ]:
colors = pd.Series(
    ["red", "purple", "blue", "green", "yellow"],
    index=[1, 2, 3, 5, 8]
)
colors
# Expected:
# 1       red
# 2    purple
# 3      blue
# 5     green
# 8    yellow
# dtype: object

Die Pandas Funktionen `loc[i]` und  `iloc[j]` können zusätzlich benutzt werden um auf Elemente von Series zuzugreifen. 

* `loc[i]` liefert das Element der Series mit dem Index `i` zurück. Es wird vom expliziten Index gesprochen.
* `iloc[j]` liefert das Element an der j-ten Position zurück. Es wird hier vom impliziten Index gesprochen.

Es sind auch Bereiche (wie oben) möglich.


In [ ]:
colors.loc[1]
# Expected:
# 'red'

In [ ]:
colors.iloc[1]
# Expected:
# 'purple'

In [ ]:
colors.iloc[1:3]
# Expected:
# 2    purple
# 3      blue
# dtype: object

In [ ]:
colors.loc[3:8]
# Expected:
# 3      blue
# 5     green
# 8    yellow
# dtype: object

In [ ]:
colors.iloc[-2]
# Expected:
# 'green'

## Zugriff auf `DataFrame` Elemente

### Zugriff auf Spalten eines `DataFrame`

Da ein DataFrame zwei Dimensionen besitzt gibt es nun mehrere Möglichkeiten Elemente aus diesem zu selektieren. 

Sie können entweder 
* einen Teil der Spalten,
* einen Teil der Zeilen oder
* eine Kombination aus Zeilen und Spalten 

des DataFrames selektieren.

Für die Spalten verwenden Sie den klassischen Python Index Operator `[]` mit dem Indexwert für die jeweilige Spalte.

In [ ]:
city_data["revenue"]
# Expected:
# Amsterdam    4200
# Tokyo        6500
# Toronto      8000
# Name: revenue, dtype: int64

Dieser jetzt eindimensionale Teil des DataFrames ist folgerichtig eine Series.

In [ ]:
type(city_data["revenue"])
# Expected:
# pandas.core.series.Series

Alternativ kann auch über die Attribute des Dataframes auf eine Spalte zugegriffen werden.

In [ ]:
city_data.revenue
# Expected:
# Amsterdam    4200
# Tokyo        6500
# Toronto      8000
# Name: revenue, dtype: int64

Hier kann es aber zu Problemen mit den Namensbereichen kommen. So ist das Attribut `shape` ein allgemeines Attribut das die Ausmaße, sprich die Form des DataFrames beschreibt. Eine gleichnamige Spalte kann also so nicht angesprochen werden, das der Name bereits belegt ist. Aus sind so Zugriffe auf Spalten mit Sonderzeichen in den Indexwerten nicht möglich.

In [ ]:
toys = pd.DataFrame([
    {"name": "ball", "shape": "sphere"},
    {"name": "Rubik's cube", "shape": "cube"}
])
toys["shape"]
# 0    sphere
# 1      cube
# Name: shape, dtype: object

In [ ]:
toys.shape
# Expected:
# (2, 2)

### Zugriff auf Zeilen des `DataFrame` mit `.loc` und `.iloc`

`iloc` und `loc` selektieren Zeilen des Dataframes iwe oben beschrieben entweder explizit (`loc`) oder implizit (`iloc`). Wird nur eine Zeile selektiert so entsteht eine Series, werden mehrere Zeilen selektiert so ist das Ergebnis erneut ein DataFrame.

In [ ]:
city_data.loc["Amsterdam"]
# Expected:
# revenue           4200.0
# employee_count       5.0
# Name: Amsterdam, dtype: float64

In [ ]:
city_data.loc["Tokyo": "Toronto"]
# Expected:
# revenue employee_count
# Tokyo   6500    8.0
# Toronto 8000    NaN

In [ ]:
city_data.iloc[1]
# Expected:
# revenue           6500.0
# employee_count       8.0
# Name: Tokyo, dtype: float64

#### Aufgabe

Geben Sie die vorletzte Zeile des nba datasets aus.

### Zugriff auf eine Teilmenge von Zeilen `und` Spalten eines `DataFrame` mit `.loc` und `.iloc`

Indem man `loc` eine durch Komma getrennte Kombination aus Indexwerten für beide Achsen übergibt, erhält man bestimmte Spaten für einen Teil der Zeilen. Hier wird im Zeilenindex ein Wertebereich und für den Spaltenindex ein Einzelwert ausgewählt.

In [ ]:
city_data.loc["Amsterdam": "Tokyo",  "revenue"]
# Expected:
# Amsterdam    4200
# Tokyo        6500
# Name: revenue, dtype: int64

In [ ]:
city_data.loc[["Amsterdam", "Toronto"],"revenue"]
# Expected:
# Amsterdam    4200
# Toronto      8000
# Name: revenue, dtype: int64

#### Aufgabe

Schauen Sie sich die Spiele mit den Labels 5555 und 5559 an. Wir interessieren uns für die Namen der beteiligten Mannschaften und die jeweils erzielten Punkte.


## Abfragen mit Pandas

Nicht immer sind direkte Selektionen möglich oder gewollt. Oft muss ein Bereich nach einem bestimmten Kriterium oder einer Kombination von Kriterien durchsucht werden. Dazu können Sie direkt nach dem Namens des DataFrame in eckigen Klammern eine Reihe von Bedingungen angeben nach denen die zu selektierenden Zeilen ausgewählt werden.

In [ ]:
nba[nba["year_id"] > 2010]
# Expected:
# 12658 rows × 24 columns

Es können neue DataFrames für die spätere Verwendung auch auf Basis einer Selektion auf einen bestehenden DataFrame angelegt werden. Dies ist für Zwischenergebnisse oft sehr sinnvoll.

In [ ]:
games_with_notes = nba[nba["notes"].notnull()]
games_with_notes.shape
# Expected:
# (5424, 24)

In [ ]:
ers = nba[nba["fran_id"].str.endswith("ers")]
ers.shape
# Expected:
# (27797, 24)

Sollen anschließend noch bestimmte Spalten oder gar explizit Zeilen ausgeschlossen werden, so kann die `loc`-Funktion angehängt werden.

In [ ]:
nba[
    (nba["_iscopy"] == 0) &
    (nba["pts"] > 100) &
    (nba["opp_pts"] > 100) &
    (nba["team_id"] == "BLB")
].loc[:,"fran_id"]
# Expected:
# 1726    Baltimore
# 4890    Baltimore
# 4909    Baltimore
# 5208    Baltimore
# 5825    Baltimore
# Name: fran_id, dtype: object

#### Aufgabe

Im Frühjahr des Jahres 1992 mussten beide Teams aus Los Angeles ein Heimspiel an einem anderen Ort abhalten. (Beide Spiele haben eine ID die mit "LA" beginnt. Änderungen und Bemerkungen sind in der Spalte notes vermerkt)

[Hintergrund](https://de.wikipedia.org/wiki/Unruhen_in_Los_Angeles_1992)

## Aggregationsfunktionen

Aggregationsfunktionen werden verwendet um zusammenfassende Berechnungen über mehrere Werte oder sogar ganze Spalten durchzuführen. 
Sie sind in der Regel auf numerische WErte beschränkt.

Gängige Funktionen sind:
* Min
* Max
* Count
* Sum

### Aggregationen für `Series`

In [ ]:
city_revenues.sum()
# Expected:
# 18700

In [ ]:
city_revenues.max()
# Expected:
# 8000

### Aggregationen für `DataFrames`

Diese funktionieren analog zu Series, jedoch ist in den meisten Fällen die Einschränkung auf ein oder mehrere Spalten sinnvoll.


In [ ]:
points = nba["pts"]
type(points)
# Expected:
# <class 'pandas.core.series.Series'>

In [ ]:
points.sum()
# Expected:
# 12976235

Das funktioniert natürlich auch ohne Zwischenspeichern:

In [ ]:
nba["pts"].sum()

### Grouping (Gruppen)

Sollen nun die gesammelten Punkte für jedes Team separat und nicht für die gesamte Liga aufsummiert werden, muss dei `groupby`-Funktion verwendet werden.
Dabei werden die Gruppen (also die Teams) während der Laufzeit aus den Daten ermittelt und die Punkte auf einem "Punktekonto" für jedes gefundene Team gesammelt.

In [ ]:
nba.groupby("fran_id", sort=False)["pts"].sum()
# Expected:
# fran_id
# Huskies           3995
# Knicks          582497
# Stags            20398
# Falcons           3797
# Capitols         22387

Es sind auch mehrstufige Gruppierungen möglich. Im Beispiel unten wird pro Jahr die Anzahl der gewonnenen und verlorenen Spiele ermittelt.

Sie sehen zusätzlich sehr schön wie die Abfrage vorher auf einen Zeitbereich und ein Team eingeschränkt wird.

In [ ]:
nba[
    (nba["fran_id"] == "Spurs") &
    (nba["year_id"] > 2010)
].groupby(["year_id", "game_result"])["game_id"].count()
# Expected:
# year_id  game_result
# 2011     L              25
# W              63
# 2012     L              20
# W              60
# 2013     L              30
# W              73
# 2014     L              27
# W              78
# 2015     L              31
# W              58
# Name: game_id, dtype: int64

#### Aufgabe

Sehen wir uns die Saison 2014-15 (year_id: 2015) der Golden State Warriors genauer an. Wie viele Siege und Niederlagen gab es in der regulären Saison, wie viele in den Playoffs (Spalte is_playoffs)? 


## Änderungen an DataFrames

DataFrames können 1:1 kopiert werden.

In [ ]:
df = nba.copy()
df.shape
# Expected:
# (126314, 24)

### Hinzufügen neuer Spalten

Hier wird die Punktdifferenz als Ergebnis einer elementweisen Subtraktion zwischen zwei Spalten in eine neue Spalte gespeichert.

In [ ]:
df["difference"] = df.pts - df.opp_pts
df.shape
# Expected:
# (126314, 25)

In [ ]:
df["difference"].max()
# Expected:
# 68

### Umbenennen von Spalten

Spalten können beim Kopieren von DataFrames aber auch bei bestehenden DataFrames umbenannt werden., im letzteren Fall muss der Parameter `inplace = true` gesetzt werden.

In [ ]:
renamed_df = df.rename(
    columns={"game_result": "result", "game_location": "location"}
)
renamed_df.info()
# Expected:
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 126314 entries, 0 to 126313
# Data columns (total 25 columns):
# gameorder      126314 non-null int64

# location       126314 non-null object
# result         126314 non-null object
# forecast       126314 non-null float64
# notes          5424 non-null object
# date_played    126314 non-null datetime64[ns]
# difference     126314 non-null int64
# dtypes: datetime64[ns](1), float64(6), int64(8), object(10)
# memory usage: 24.1+ MB

### Spalten löschen

In [ ]:
df.shape
# Expected:
# (126314, 25)

Im Beispiel wird eine Liste an zu löschenden Spalten vorbereitet. Die Funktion `drop()` löscht nun aus den Spalten (`axis=1`), direkt im DataFrame (`inplace=true`) die angegebenen Spalten.

Auf die gleiche Weise können auch Zeilen aus dem DataFrame gelöscht werden. Dazu muss nur auf dem Zeilenindex (`axis=0`) und mit passender Indexliste für die Zeilen gearbeitet werden. 

In [ ]:
elo_columns = ["elo_i", "elo_n", "opp_elo_i", "opp_elo_n"]
df.drop(elo_columns, inplace=True, axis=1)
df.shape
# Expected:
# (126314, 21)

### Datentyp einer Spalte ändern

Der Datentyp einer Spalte kann(!) beim Initialisieren des DataFrames festgelegt werden. Wird das nicht gemacht entstehen of Spalten mit sehr generischen Datentypen wie object. 

So ist in unserem BEspiel die Spalte date_game vpn Typ object obwohl dort sicher ein Datum gespeichert ist. Funktionen zum Verarbeiten von Datumswerten können hier also nicht ohne weiteres angewendet werden. Deeswegen macht es oft Sinn diese Spalten umzuwandeln.

In [ ]:
df.info()

Hier wird nun die Spalte mit der Pandas Funktion `to_datetime` umgewandelt und anschließen an der vorigen Platz kopiert.

In [ ]:
df["date_game"] = pd.to_datetime(df["date_game"])

Sollten Spalten nur einige wenige unterschiedliche Werte enthalten, so handelt sich of um kategoriale Variablen. Diese sollten im entsprechenden Datentyp abgelegt sein so dass diese a) speicherschonend verarbeitet werden und b) für statistische Auswertungen passend weiterverarbeitet werden können. 

In [ ]:
df["game_location"].nunique()
# Expected:
# 3

In [ ]:
df["game_location"].value_counts()
# Expected:
# A    63138
# H    63138
# N       38
# Name: game_location, dtype: int64

In [ ]:
df["game_location"] = pd.Categorical(df["game_location"])
df["game_location"].dtype
# Expected:
# CategoricalDtype(categories=['A', 'H', 'N'], ordered=False)

#### Aufgabe

Finden Sie eine weitere Spalte, die einen zu allgemeinen Datentyp hat und ändern Sie diesen.

## Erste Visualisierungen
In Pandas sind einige Visualisierungsfunktionalitäten direkt eingebaut und können über die Funktion `plot()` direkt aufgerufen werden.

In [ ]:
%matplotlib inline

Hier wird die Summe der erzielten Punkte pro Jahr (--> `groupby`) für die New York Knicks ermittelt. Das ERgebnis ist eine Pandas Series die sich ein einem Liniendiagramm visualisieren lässt.

In [ ]:
nba[nba["fran_id"] == "Knicks"].groupby("year_id")["pts"].sum().plot()

Die Funktion `value_counts()` liefert für eine Series die Anzahl der eindeutigen Werte zurück. die zehn häufigsten Werte werden nun per Säulendiagramm visualisiert.

In [ ]:
nba["fran_id"].value_counts().head(10).plot(kind="bar")

#### Aufgabe

In 2013 haben die Miami Heat die Meisterschaft gewonnen. Erzeugen Sie ein Kuchendiagramm (= Pie) das die Anzahl der Siege und Niederlagen in dieser Saison zeigt. 
